In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

from pathlib import Path  
import glob

In [4]:
# directory_path = "/Users/diliyayalikun/Downloads/New v copy/src/data/earnings_calls"
directory_path = "/Users/diliyayalikun/Downloads/New v copy/src/data/earnings_calls/2016"

In [5]:
text_files = glob.glob(f"{directory_path}/*.xml")
text_files

['/Users/diliyayalikun/Downloads/New v copy/src/data/earnings_calls/2016/7331107_T.xml',
 '/Users/diliyayalikun/Downloads/New v copy/src/data/earnings_calls/2016/7307951_T.xml',
 '/Users/diliyayalikun/Downloads/New v copy/src/data/earnings_calls/2016/7330709_T.xml',
 '/Users/diliyayalikun/Downloads/New v copy/src/data/earnings_calls/2016/7331098_T.xml',
 '/Users/diliyayalikun/Downloads/New v copy/src/data/earnings_calls/2016/7328508_T.xml',
 '/Users/diliyayalikun/Downloads/New v copy/src/data/earnings_calls/2016/7325241_T.xml',
 '/Users/diliyayalikun/Downloads/New v copy/src/data/earnings_calls/2016/7332866_T.xml',
 '/Users/diliyayalikun/Downloads/New v copy/src/data/earnings_calls/2016/7330148_T.xml',
 '/Users/diliyayalikun/Downloads/New v copy/src/data/earnings_calls/2016/7325240_T.xml',
 '/Users/diliyayalikun/Downloads/New v copy/src/data/earnings_calls/2016/7328416_T.xml',
 '/Users/diliyayalikun/Downloads/New v copy/src/data/earnings_calls/2016/7308156_T.xml',
 '/Users/diliyayaliku

In [6]:
text_titles = [Path(text).stem for text in text_files]
text_titles

['7331107_T',
 '7307951_T',
 '7330709_T',
 '7331098_T',
 '7328508_T',
 '7325241_T',
 '7332866_T',
 '7330148_T',
 '7325240_T',
 '7328416_T',
 '7308156_T',
 '7325242_T',
 '7307299_T']

In [16]:
tfidf_vectorizer = TfidfVectorizer(input='filename', stop_words='english',ngram_range=(1,2))
tfidf_vector = tfidf_vectorizer.fit_transform(text_files)
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=text_titles, columns=tfidf_vectorizer.get_feature_names_out())

In [8]:
# tfidf_df.loc['00_Document Frequency'] = (tfidf_df > 0).sum()
# tfidf_slice = tfidf_df[['earning', 'transcript', 'increase', 'conference', 'lost', 'stocks','tax', 'men', 'women', 'children']]
# tfidf_slice.sort_index().round(decimals=2)

In [9]:
# tfidf_df = tfidf_df.drop('00_Document Frequency', errors='ignore')

In [17]:

tfidf_df.stack().reset_index()
# tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})
# tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)

,level_0,level_1,0
0,7331107_T,00,0.013602
1,7331107_T,00 00am,0.000000
2,7331107_T,00 00pm,0.000000
3,7331107_T,00 10pm,0.000000
4,7331107_T,00 inaudible,0.010846
...,...,...,...
380284,7307299_T,zone data,0.000000
380285,7307299_T,zone rar,0.000000
380286,7307299_T,zone really,0.000000
380287,7307299_T,zurcher,0.000000


In [18]:
tfidf_df = tfidf_df.stack().reset_index()
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})

In [19]:
tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)

,document,term,tfidf
366583,7307299_T,lloyds,0.267734
354473,7307299_T,banking,0.235606
354474,7307299_T,banking group,0.235606
366585,7307299_T,lloyds banking,0.235606
363245,7307299_T,group plc,0.230251
...,...,...,...
193590,7332866_T,nv,0.190753
189852,7332866_T,john porter,0.142540
195447,7332866_T,porter,0.142540
193282,7332866_T,network,0.129067


In [20]:
top_tfidf = tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)
top_tfidf[top_tfidf['term'].str.contains('b2b')]

,document,term,tfidf


In [21]:
top_tfidf[top_tfidf['document'].str.contains('7330148')]

,document,term,tfidf
227474,7330148_T,rh,0.289695
226003,7330148_T,quarter,0.190287
215629,7330148_T,fiscal,0.154652
233739,7330148_T,year,0.113497
215630,7330148_T,fiscal 2016,0.110360
221531,7330148_T,membership,0.110360
216128,7330148_T,galleries,0.096565
217941,7330148_T,impacting,0.096565
216946,7330148_T,gross margin,0.095171
208962,7330148_T,brand,0.094943


In [22]:
import altair as alt
import numpy as np

# Terms in this list will get a red dot in the visualization
term_list = ['war', 'peace']

# adding a little randomness to break ties in term ranking
top_tfidf_plusRand = top_tfidf.copy()
top_tfidf_plusRand['tfidf'] = top_tfidf_plusRand['tfidf'] + np.random.rand(top_tfidf.shape[0])*0.0001

# base for all visualizations, with rank calculation
base = alt.Chart(top_tfidf_plusRand).encode(
    x = 'rank:O',
    y = 'document:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["document"],
)

# heatmap specification
heatmap = base.mark_rect().encode(
    color = 'tfidf:Q'
)

# red circle over terms in above list
circle = base.mark_circle(size=100).encode(
    color = alt.condition(
        alt.FieldOneOfPredicate(field='term', oneOf=term_list),
        alt.value('red'),
        alt.value('#FFFFFF00')        
    )
)

# text labels, white for darker heatmap colors
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)

# display the three superimposed visualizations
(heatmap + circle + text).properties(width = 600)

alt.LayerChart(...)